In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.optim import Adam
import random
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
import math
from torch.utils.data import TensorDataset, DataLoader
import os

# 시드 고정
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seed(42)

In [2]:
data = pd.read_csv('/home/aibig25/hong_sj/trb/num.csv')
data = data.fillna(0)

unique_ids = data['sequence_ID'].unique()
train_ids, test_ids = train_test_split(unique_ids, test_size=41, random_state=42)
train_data = data[data['sequence_ID'].isin(train_ids)]
test_data = data[data['sequence_ID'].isin(test_ids)]

independent_vars = data.columns.difference(['center_x', 'center_y','center_x_ma','center_y_ma', 'ID', 'LC'])
dependent_vars = ['center_y_ma']

scaler = MinMaxScaler()

train_data[independent_vars] = scaler.fit_transform(train_data[independent_vars])
test_data[independent_vars] = scaler.transform(test_data[independent_vars])

X_train = train_data[independent_vars]
y_train = train_data[dependent_vars]

X_test = test_data[independent_vars]
y_test = test_data[dependent_vars]

/tmp/ipykernel_100102/3224682186.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data[independent_vars] = scaler.fit_transform(train_data[independent_vars])
/tmp/ipykernel_100102/3224682186.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data[independent_vars] = scaler.transform(test_data[independent_vars])


In [3]:
# 입력 및 예측 시퀀스 길이 정의
input_sequence_length = 60
output_sequence_length = 30

def create_sequences(data, input_sequence_length, output_sequence_length):
    X = []
    y = []

    for i in range(len(data) - input_sequence_length - output_sequence_length + 1):
        X.append(data.iloc[i:(i + input_sequence_length)][independent_vars].values)
        y.append(data.iloc[(i + input_sequence_length):(i + input_sequence_length + output_sequence_length)][dependent_vars].values)
    
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_data, input_sequence_length, output_sequence_length)
X_test, y_test = create_sequences(test_data, input_sequence_length, output_sequence_length)

In [4]:
# 데이터셋을 텐서로 변환
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32))

# 데이터 로더 생성
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

In [5]:
class TrajectoryTransformer(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_encoder_layers, num_decoder_layers, output_dim, lstm_hidden_dim, lstm_layers=1):
        super(TrajectoryTransformer, self).__init__()
        self.model_dim = model_dim
        
        # Linear layer to transform input dimensions to model dimensions
        self.encoder = nn.Linear(input_dim, model_dim)
        self.pos_encoder = PositionalEncoding(model_dim)
        self.tgt_linear = nn.Linear(1, model_dim)  # Linear layer for transforming tgt dimensions
        
        # Define transformer encoder and decoder
        self.encoder_layers = nn.ModuleList([
            TransformerEncoderLayer(model_dim, num_heads, lstm_hidden_dim, lstm_layers)
            for _ in range(num_encoder_layers)
        ])
        self.decoder_layers = nn.ModuleList([
            TransformerDecoderLayer(model_dim, num_heads, lstm_hidden_dim, lstm_layers)
            for _ in range(num_decoder_layers)
        ])
        
        self.decoder = nn.Linear(model_dim, output_dim)

    def forward(self, src, tgt):
        # Encoding source input
        src = self.encoder(src)
        src = src * math.sqrt(self.model_dim)
        src = self.pos_encoder(src.permute(1, 0, 2))  # Add positional encoding

        # Processing target input
        tgt = tgt.squeeze(-1)
        original_shape = tgt.shape
        tgt = tgt.reshape(-1, 1)
        tgt = self.tgt_linear(tgt)
        tgt = tgt.view(original_shape[0], original_shape[1], -1)
        tgt = tgt * math.sqrt(self.model_dim)
        tgt = self.pos_encoder(tgt.permute(1, 0, 2))  # Add positional encoding

        # Apply encoder and decoder layers
        for layer in self.encoder_layers:
            src = layer(src)
        
        for layer in self.decoder_layers:
            tgt = layer(tgt, src)

        # Decode output and return
        output = self.decoder(tgt.permute(1, 0, 2))
        return output

class TransformerEncoderLayer(nn.Module):
    def __init__(self, model_dim, num_heads, lstm_hidden_dim, lstm_layers):
        super(TransformerEncoderLayer, self).__init__()
        self.self_attn = nn.MultiheadAttention(model_dim, num_heads, dropout=0.1)
        
        # LSTM instead of Feedforward layer
        self.lstm = nn.LSTM(input_size=model_dim, hidden_size=lstm_hidden_dim, num_layers=lstm_layers, batch_first=True, dropout=0.1)
        self.linear = nn.Linear(lstm_hidden_dim, model_dim)
        
        self.norm1 = nn.LayerNorm(model_dim)
        self.norm2 = nn.LayerNorm(model_dim)
        self.dropout = nn.Dropout(0.1)

    def forward(self, src):
        # Self-attention
        src2 = self.self_attn(src, src, src)[0]
        src = src + self.dropout(src2)
        src = self.norm1(src)
        
        # LSTM
        lstm_out, _ = self.lstm(src.permute(1, 0, 2))  # LSTM requires (batch_size, seq_length, input_size)
        lstm_out = lstm_out.permute(1, 0, 2)  # Transpose back to (seq_length, batch_size, input_size)
        
        # Linear transformation
        src2 = self.linear(lstm_out)
        src = src + self.dropout(src2)
        src = self.norm2(src)
        return src

class TransformerDecoderLayer(nn.Module):
    def __init__(self, model_dim, num_heads, lstm_hidden_dim, lstm_layers):
        super(TransformerDecoderLayer, self).__init__()
        self.self_attn = nn.MultiheadAttention(model_dim, num_heads, dropout=0.1)
        self.multihead_attn = nn.MultiheadAttention(model_dim, num_heads, dropout=0.1)
        
        # LSTM instead of Feedforward layer
        self.lstm = nn.LSTM(input_size=model_dim, hidden_size=lstm_hidden_dim, num_layers=lstm_layers, batch_first=True, dropout=0.1)
        self.linear = nn.Linear(lstm_hidden_dim, model_dim)
        
        self.norm1 = nn.LayerNorm(model_dim)
        self.norm2 = nn.LayerNorm(model_dim)
        self.norm3 = nn.LayerNorm(model_dim)
        self.dropout = nn.Dropout(0.1)

    def forward(self, tgt, memory):
        # Self-attention for the target sequence
        tgt2 = self.self_attn(tgt, tgt, tgt)[0]
        tgt = tgt + self.dropout(tgt2)
        tgt = self.norm1(tgt)
        
        # Cross-attention with encoder memory
        tgt2 = self.multihead_attn(tgt, memory, memory)[0]
        tgt = tgt + self.dropout(tgt2)
        tgt = self.norm2(tgt)
        
        # LSTM
        lstm_out, _ = self.lstm(tgt.permute(1, 0, 2))  # LSTM requires (batch_size, seq_length, input_size)
        lstm_out = lstm_out.permute(1, 0, 2)  # Transpose back to (seq_length, batch_size, input_size)
        
        # Linear transformation
        tgt2 = self.linear(lstm_out)
        tgt = tgt + self.dropout(tgt2)
        tgt = self.norm3(tgt)
        return tgt

# 위치 인코딩 추가
class PositionalEncoding(nn.Module):
    def __init__(self, model_dim, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, model_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, model_dim, 2).float() * (-math.log(10000.0) / model_dim))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = self.encoding.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', self.encoding)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return x


In [6]:
# 모델 초기화
input_dim = len(independent_vars)
output_dim = len(dependent_vars)
model_dim = 160
num_heads = 5
num_encoder_layers = 3
num_decoder_layers = 3
lstm_hidden_dim = 50
lstm_layers = 1

model = TrajectoryTransformer(input_dim, model_dim, num_heads, num_encoder_layers, num_decoder_layers, output_dim, lstm_hidden_dim, lstm_layers)

# 옵티마이저와 손실 함수
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

/home/aibig25/anaconda3/envs/solar/lib/python3.8/site-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [7]:
def load_model(model_path, model_class, *model_args, **model_kwargs):
    model = model_class(*model_args, **model_kwargs)
    model.load_state_dict(torch.load(model_path))
    model.eval()  # 평가 모드로 설정
    return model

def get_attention_weights(model, src, tgt):
    attention_weights = []
    
    src = model.encoder(src)
    src = src * math.sqrt(model.model_dim)
    src = model.pos_encoder(src.permute(1, 0, 2))

    tgt = tgt.squeeze(-1)
    original_shape = tgt.shape
    tgt = tgt.reshape(-1, 1)
    tgt = model.tgt_linear(tgt)
    tgt = tgt.view(original_shape[0], original_shape[1], -1)
    tgt = tgt * math.sqrt(model.model_dim)
    tgt = model.pos_encoder(tgt.permute(1, 0, 2))

    for layer in model.encoder_layers:
        src = layer(src)
        attention_weights.append(layer.self_attn(src, src, src)[1].detach().cpu().numpy())
    
    for layer in model.decoder_layers:
        tgt = layer(tgt, src)
        attention_weights.append(layer.self_attn(tgt, tgt, tgt)[1].detach().cpu().numpy())

    return attention_weights

def plot_attention_heatmap(attention_weights, input_seq_num, input_seq_length, output_seq_length, title, save_path=None):
    avg_attention_weights = np.mean(attention_weights, axis=0).squeeze()

    if avg_attention_weights.ndim == 3:
        avg_attention_weights = avg_attention_weights.mean(axis=0)

    # input sequence가 x축에, output sequence가 y축에 오도록 설정
    plt.figure(figsize=(10, 8))
    sns.heatmap(avg_attention_weights[:output_seq_length, :input_seq_length], cmap='viridis', 
                xticklabels=range(1, input_seq_length + 1), 
                yticklabels=range(1, output_seq_length + 1))
    plt.title(f'{title} (Input Sequence {input_seq_num})')
    plt.xlabel('Input Sequence Position')
    plt.ylabel('Output Sequence Position')
    
    if save_path:
        plt.savefig(save_path)
        plt.close()
    else:
        plt.show()

def visualize_attention_for_sequence(model, dataset, sequence_id, input_sequence_length, output_sequence_length, save_dir=None):
    sequence_data = dataset[dataset['sequence_ID'] == sequence_id]
    
    X, y = create_sequences(sequence_data, input_sequence_length, output_sequence_length)
    
    src_tensor = torch.tensor(X, dtype=torch.float32)
    tgt_tensor = torch.tensor(y, dtype=torch.float32)

    if save_dir:
        os.makedirs(save_dir, exist_ok=True)

    for i in range(src_tensor.size(0)):
        src = src_tensor[i:i+1]
        tgt = tgt_tensor[i:i+1]

        attention_weights = get_attention_weights(model, src, tgt)
        
        if save_dir:
            save_path = os.path.join(save_dir, f'seq_{sequence_id}_input_{i+1}.png')
            plot_attention_heatmap(attention_weights[0], i + 1, input_sequence_length, output_sequence_length, 
                                   f'Attention Heatmap for Sequence ID {sequence_id}', save_path=save_path)
        else:
            plot_attention_heatmap(attention_weights[0], i + 1, input_sequence_length, output_sequence_length, 
                                   f'Attention Heatmap for Sequence ID {sequence_id}')

In [8]:
# 학습된 모델 로드
model_path = "/home/aibig25/hong_sj/trb/transformer/new/code/2_1.pth"  
model = load_model(model_path, TrajectoryTransformer, 
                   input_dim, model_dim, num_heads, num_encoder_layers, num_decoder_layers, output_dim, lstm_hidden_dim, lstm_layers)

In [13]:
sequence_id = 78
save_directory = f'/home/aibig25/hong_sj/trb/transformer/new/heatmap/average/2 to 1/{sequence_id}'  
visualize_attention_for_sequence(model, data, sequence_id, input_sequence_length, output_sequence_length, save_dir=save_directory)